In [1]:
import carla
import random

In [2]:
# Connect to the client and retrieve the world object
client = carla.Client('localhost', 2000)
world = client.get_world()

In [3]:
client.load_world('Town03')

In [4]:
# Get the blueprint library and filter for the vehicle blueprints
vehicle_blueprints = world.get_blueprint_library().filter('*vehicle*')
# Get the map's spawn points
spawn_points = world.get_map().get_spawn_points()

In [5]:
# Spawn 50 vehicles randomly distributed throughout the map 
# for each spawn point, we choose a random vehicle from the blueprint library
for i in range(0,50):
    world.try_spawn_actor(random.choice(vehicle_blueprints), random.choice(spawn_points))

In [6]:
for vehicle in world.get_actors().filter('*vehicle*'):
    vehicle.set_autopilot(True)

In [7]:
ego_bp = world.get_blueprint_library().find('vehicle.seat.leon')

ego_bp.set_attribute('role_name', 'hero')

ego_vehicle = world.spawn_actor(ego_bp, random.choice(spawn_points))

In [8]:
# Create a transform to place the camera on top of the vehicle
camera_init_trans = carla.Transform(carla.Location(z=1.5))

# We create the camera through a blueprint that defines its properties
camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')

# We spawn the camera and attach it to our ego vehicle
camera = world.spawn_actor(camera_bp, camera_init_trans, attach_to=ego_vehicle)

In [9]:
import cv2
import numpy as np

def show_image(image):
    # Преобразуем CARLA image в массив numpy
    array = np.frombuffer(image.raw_data, dtype=np.uint8)
    array = array.reshape((image.height, image.width, 4))  # BGRA формат

    # Избавляемся от альфа-канала (если не нужен)
    frame = array[:, :, :3]

    # Преобразуем BGRA -> RGB, если нужно (OpenCV по умолчанию в BGR)
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Показываем изображение
    cv2.imshow("CARLA Camera", frame)
    
    # Обновляем окно; 1 мс ожидания клавиши (нужно для обработки GUI)
    if cv2.waitKey(1) == 27:  # Нажми Esc чтобы выйти
        image.stop()


In [48]:
# Пример подключения:
camera.listen(show_image)

: 

In [ ]:
# Управление: газ, поворот, тормоз
control = carla.VehicleControl()

In [45]:
control.steer = +0.00     # Поворот от -1.0 до 1.0
ego_vehicle.apply_control(control)

In [46]:
control.throttle = 0.5  # Газ от 0.0 до 1.0
control.brake = 0.0
ego_vehicle.apply_control(control)

In [47]:
# Остановим
control.throttle = 0.0
control.brake = 1.0
ego_vehicle.apply_control(control)